In [59]:
import pandas as pd
import numpy as np

import datetime
import pickle

import openpyxl

from IPython.display import display, HTML
import base64


In [60]:
shop_DE = pd.read_pickle("shop_de.pkl")
shop_DE.head(1)

,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE
0,January,15562.55,14774.46,148945.17,179282.18


In [61]:
shop_EU = pd.read_pickle("shop_eu.pkl")
shop_EU.head(1)

,Monat,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,1514.91,1572.0,14350.5,17437.41


In [62]:
el_puente= pd.merge(shop_DE,shop_EU, on="Monat")
el_puente

,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU
0,January,15562.55,14774.46,148945.17,179282.18,1514.91,1572.0,14350.50,17437.41
1,February,14055.57,15176.47,128859.95,158091.99,1495.04,1430.0,9155.79,12080.83
2,March,15633.91,18280.33,141602.71,175516.95,1789.00,1317.0,12937.99,16044.00
3,April,15792.85,19095.32,133780.49,168668.66,1392.16,1530.0,14028.52,16950.69
4,May,16947.49,20206.11,135993.78,173147.38,1719.61,1435.0,10523.91,13678.51
5,June,17082.75,20056.65,135206.77,172346.17,1588.09,1392.0,12650.49,15630.58
6,July,17965.03,22288.14,143531.78,183784.95,1813.79,1710.0,9448.17,12971.96
7,August,17720.17,21707.81,139714.63,179142.61,1588.00,1713.0,12392.66,15693.66
8,September,16870.28,21084.84,130178.03,168133.15,1495.48,1632.0,12147.95,15275.43
9,October,17579.24,21564.91,128966.72,168110.87,1836.13,1502.0,13321.29,16659.42


In [63]:
el_puente.to_excel("El_Puente.xlsx", engine='openpyxl', float_format='%.2f')

In [64]:
el_puente.to_pickle("total.pkl")


In [65]:
df = pd.read_pickle("total.pkl")

In [66]:
df =df.drop(["Gesamt_DE", "Gesamt_EU"], axis=1)

In [67]:
df["Monat"].replace({"January":"Jan", "February":"Feb", "March":"Mar", "April":"Apr",
                            "May":"Mai", "June":"Jun", "July":"Jul", "August":"Aug", "September":"Sep",
                            "October":"Okt", "November":"Nov", "December":"Dez"}, inplace=True)

In [68]:
shop_ohne_gesamt = pd.melt(df, id_vars='Monat', var_name='Produkt', value_name='Umsatz')
shop_ohne_gesamt

,Monat,Produkt,Umsatz
0,Jan,Produkt_A_DE,15562.55
1,Feb,Produkt_A_DE,14055.57
2,Mar,Produkt_A_DE,15633.91
3,Apr,Produkt_A_DE,15792.85
4,Mai,Produkt_A_DE,16947.49
...,...,...,...
67,Aug,Produkt_C_EU,12392.66
68,Sep,Produkt_C_EU,12147.95
69,Okt,Produkt_C_EU,13321.29
70,Nov,Produkt_C_EU,11309.23


In [69]:
shop_ohne_gesamt.to_pickle("total_melt.pkl")

In [85]:
"""DataFrame als tabelle für Presentation vorbereiten - Gesamt als Spalte und Zeile hinzufügen"""
tabelle = el_puente
tabelle["TOTAL"] = tabelle["Gesamt_DE"]+tabelle["Gesamt_EU"]
tabelle.loc["Summen"] = tabelle.sum()
tabelle.loc["Summen"] = tabelle.loc["Summen"].drop("Monat")
tabelle.head(1)


C:\Users\User\AppData\Local\Temp\ipykernel_6728\34788434.py:4: FutureWarning: The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  tabelle.loc["Summen"] = tabelle.sum()


,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU,TOTAL
0,January,15562.55,14774.46,148945.17,179282.18,1514.91,1572.0,14350.5,17437.41,196719.59


In [86]:
tabelle.to_excel("Tabelle.xlsx", engine='openpyxl', float_format='%.2f')

In [82]:
""" tabelle mit Gesamtdaten für Präsentation in browser vorbereiten"""
display(HTML(tabelle.to_html()))

tabelle.to_html('tab.html')
with open('tab.html', 'r') as file:
    html_code = file.read()

"""Gesamtsummen, "Gesamt_DE", "Gesamt_EU"  fett formatieren"""
html_code = html_code.replace('<td>Gesamt_DE</td>', '<td><b>Gesamt_DE</b></td>')
html_code = html_code.replace('<td>Gesamt_EU</td>', '<td><b>Gesamt_EU</b></td>')
html_code = html_code.replace('<td>TOTAL</td>', '<td><b>TOTAL</b></td>')

encoded_image = base64.b64encode(html_code.encode('utf-8')).decode('utf-8')

display(HTML(f'<img src="data:image/png;base64,{encoded_image}">'))

,Monat,Produkt_A_DE,Produkt_B_DE,Produkt_C_DE,Gesamt_DE,Produkt_A_EU,Produkt_B_EU,Produkt_C_EU,Gesamt_EU,TOTAL
0,January,15562.55,14774.46,148945.17,179282.18,1514.91,1572.0,14350.50,17437.41,196719.59
1,February,14055.57,15176.47,128859.95,158091.99,1495.04,1430.0,9155.79,12080.83,170172.82
2,March,15633.91,18280.33,141602.71,175516.95,1789.00,1317.0,12937.99,16044.00,191560.95
3,April,15792.85,19095.32,133780.49,168668.66,1392.16,1530.0,14028.52,16950.69,185619.35
4,May,16947.49,20206.11,135993.78,173147.38,1719.61,1435.0,10523.91,13678.51,186825.89
5,June,17082.75,20056.65,135206.77,172346.17,1588.09,1392.0,12650.49,15630.58,187976.75
6,July,17965.03,22288.14,143531.78,183784.95,1813.79,1710.0,9448.17,12971.96,196756.91
7,August,17720.17,21707.81,139714.63,179142.61,1588.00,1713.0,12392.66,15693.66,194836.27
8,September,16870.28,21084.84,130178.03,168133.15,1495.48,1632.0,12147.95,15275.43,183408.58
9,October,17579.24,21564.91,128966.72,168110.87,1836.13,1502.0,13321.29,16659.42,184770.29
